In [42]:
from matplotlib import pyplot as plt
import zipfile
import tempfile
import requests
import os
import numpy as np
import pandas as pd

In [246]:
def get_puf_data(first_week, last_week):
    '''
    download puf files for the given weeks and concatenate the datasets
    '''
    df = pd.DataFrame()
    for i in range(first_week,last_week+1):
        y = 2020
        if i>=22:
            y=2021
        file_str = "pulse{}_puf_{}.csv".format(y,i)
        url_str = "https://www2.census.gov/programs-surveys/demo/datasets/hhp/{yr}/wk{w}/HPS_Week{w}_PUF_CSV.zip".format(yr=y,w=i)
        week_df = download_url(url_str, file_str, y, i)
        df = pd.concat([df,week_df])

    return df


def download_url(url_str,file_str, year, week, chunk_size=128):
    '''
    download puf, avoid intermediate save to disk
    '''
    df = pd.DataFrame()
    r = requests.get(url_str)
    print(r)
    #temp directory rather than file to hopefully accomodate windows users
    with tempfile.TemporaryDirectory() as td:
        f_name = os.path.join(td, 'week_{}'.format(week))
        with open(f_name, 'wb') as fh:
            for chunk in r.iter_content(chunk_size=chunk_size):
                fh.write(chunk)
        zf = zipfile.ZipFile(f_name)
        df = pd.read_csv(zf.open(file_str))
        df1 = pd.read_csv(zf.open("pulse{}_repwgt_puf_{}.csv".format(year,week)))
        zf.close()
        r.close()

    return pd.merge(df,df1,on=["SCRAM","WEEK"],how="inner")


def get_std_err(df, weight):
    #make 1d array of weight col
    obs_wgts = df[weight].to_numpy().reshape(len(df),1)
    
    #make 80d array of replicate weights
    rep_wgts = df[[i for i in df.columns if weight in i and not i == weight]].to_numpy()
    
    #return standard error of estimate
    return np.sqrt((np.sum(np.square(rep_wgts-obs_wgts),axis=1)*(4/80)))


def freq_crosstab(df, col_list, weight, critical_val=1):
    '''

    '''
    
    pt_estimates = df.groupby(col_list, as_index=True)[[i for i in df.columns if weight in i]].agg('sum')
    pt_estimates['std_err'] = get_std_err(pt_estimates, weight)
    pt_estimates['mrgn_err'] = pt_estimates.std_err * critical_val

    return pt_estimates[[weight, 'std_err','mrgn_err']].reset_index()

def full_crosstab(df, col_list, weight, proportion_level, critical_val=1):
    df1 = df[-df[col_list].isin([-99,-88]).any(axis=1)].copy()
    print(df1.rent_check.unique())
    detail = freq_crosstab(df1, col_list, weight, critical_val)
    top = freq_crosstab(df1, proportion_level, weight, critical_val)
    rv = detail.merge(top,'left',proportion_level,suffixes=('_full','_demo'))
    rv['proportions'] = rv[weight+'_full']/rv[weight+'_demo']
    return rv


In [201]:
core_df = get_puf_data(13,22)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


## Tie Out Housing 1b Table

In [ ]:
test_df = core_df[core_df.WEEK==21].copy()
test_df['rent_check'] = test_df.apply(lambda x: 4 if x.TENURE==4 else x.RENTCUR, axis=1)
test_df['race_eth'] = test_df.apply(lambda x: 5 if x.RHISPANIC==2 else x.RRACE, axis=1)
full_crosstab(test_df, ['EGENDER', 'rent_check'], 'PWEIGHT', ['EGENDER'],critical_val=1).round(2)

In [254]:
#This cell takes forever bc i didn't take the time to map efficiently
states = {1:'Alabama',2:'Alaska',4:'Arizona',5:'Arkansas',6:'California',
    8:'Colorado',9:'Connecticut',10:'Delaware',11:'District of Columbia',12:'Florida',
    13:'Georgia',15:'Hawaii',16:'Idaho',17:'Illinois',18:'Indiana',19:'Iowa',20:'Kansas',21:'Kentucky',
    22:'Louisiana',23:'Maine',24:'Maryland',25:'Massachusetts',26:'Michigan',27:'Minnesota',
    28:'Mississippi',29:'Missouri',30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',
    34:'New Jersey',35:'New Mexico',36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',
    40:'Oklahoma',41:'Oregon',42:'Pennsylvania',44:'Rhode Island',45:'South Carolina',
    46:'South Dakota',47:'Tennessee',48:'Texas',49:'Utah',50:'Vermont',51:'Virginia',
    53:'Washington',54:'West Virginia',55:'Wisconsin',56:'Wyoming'}
regions = {1:'NE',2:'S',3:'MW',4:'W'}
metros = {35620:'New York',31080:'Los Angeles',16980:'Chicago',19100:'Dallas',
          26420:'Houston',47900:'DC',33100:'Miami',37980:'Philadelphia',
          12060:'Atlanta',38060:'Phoenix',14460:'Boston',41860:'San Francisco',
          40140:'Riverside CA',19820:'Detroit',42660:'Seattle'}
races = {1:'white',2:'black',3:'Asian',4:'Other'}
rents = {1:'current',2:'behind'}

df1 = core_df.copy()
df1['metro'] = df1.apply(lambda x: metros[x.EST_MSA] if x.EST_MSA>0 else 'remove', axis=1)
df1['state'] = df1.apply(lambda x: states[x.EST_ST],axis=1)
df1['region'] = df1.apply(lambda x: regions[x.REGION],axis=1) 
df1['race'] = df1.apply(lambda x: races[x.RRACE] if x.RRACE>0 else 'remove',axis=1)
df1['rent'] = df1.apply(lambda x: rents[x.RENTCUR] if x.RENTCUR>0 else 'remove',axis=1)

## Metro Evictions, Metro Behind Rent

In [261]:

metro_evict_ct = full_crosstab(df1[df1.EVICT>0],
                         ['WEEK','metro', 'EVICT'],
                         'PWEIGHT',
                         ['WEEK','metro'],
                         critical_val=1).round(2)
metro_evict_ct[metro_evict_ct.EVICT<3]

[2]


,WEEK,metro,EVICT,PWEIGHT_full,std_err_full,mrgn_err_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions
0,13,Atlanta,1,18260.29,8961.77,8961.77,122543.87,27640.46,27640.46,0.15
1,13,Atlanta,2,35568.09,15495.62,15495.62,122543.87,27640.46,27640.46,0.29
4,13,Boston,1,14683.87,5822.37,5822.37,96242.26,17086.31,17086.31,0.15
5,13,Boston,2,12552.96,4110.86,4110.86,96242.26,17086.31,17086.31,0.13
8,13,Chicago,1,27596.05,12829.90,12829.90,308426.51,49284.31,49284.31,0.09
...,...,...,...,...,...,...,...,...,...,...
628,22,San Francisco,2,23579.43,6982.79,6982.79,136899.18,27257.19,27257.19,0.17
631,22,Seattle,1,2300.27,2537.27,2537.27,108041.88,22645.13,22645.13,0.02
632,22,Seattle,2,33186.79,13344.04,13344.04,108041.88,22645.13,22645.13,0.31
635,22,remove,1,926768.35,158923.64,158923.64,5933799.30,354512.94,354512.94,0.16


In [258]:
metro_rent_ct = full_crosstab(df1[-(df1.rent=='remove')],
                         ['WEEK','metro', 'rent'],
                         'PWEIGHT',
                         ['WEEK','metro'],
                         critical_val=1).round(2)
metro_rent_ct[metro_ct.rent=='behind']

[1 2]


,WEEK,metro,rent,PWEIGHT_full,std_err_full,mrgn_err_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions
0,13,Atlanta,behind,131138.32,28864.81,28864.81,866185.56,78548.08,78548.08,0.15
2,13,Boston,behind,98375.64,18149.23,18149.23,785186.38,56288.05,56288.05,0.13
4,13,Chicago,behind,321886.09,51643.59,51643.59,1410490.75,104589.78,104589.78,0.23
6,13,DC,behind,157980.11,25542.01,25542.01,1136709.20,76655.35,76655.35,0.14
8,13,Dallas,behind,64494.02,14084.21,14084.21,1088451.90,74544.23,74544.23,0.06
...,...,...,...,...,...,...,...,...,...,...
310,22,Phoenix,behind,109704.60,36972.23,36972.23,750766.92,71089.52,71089.52,0.15
312,22,Riverside CA,behind,185729.13,49239.43,49239.43,940384.62,98423.02,98423.02,0.20
314,22,San Francisco,behind,138488.20,27918.33,27918.33,1091156.28,95453.62,95453.62,0.13
316,22,Seattle,behind,108041.88,22645.13,22645.13,877451.94,58737.63,58737.63,0.12


In [200]:
#full_crosstab(df, ['WEEK','metro'], 'PWEIGHT', ['WEEK','metro'], critical_val=1.645).round(2)
df1 = df[df.WEEK==13].copy()
df1 = df1[df1.metro=='Atlanta']
full_crosstab(df1, ['metro','race','rent'], 'PWEIGHT', ['metro','race'], critical_val=1.645).round(2)

,metro,race,rent,PWEIGHT,std_err,mrgn_err,proportions
0,Atlanta,Asian,behind,5923.87,3624.72,5962.66,0.22
1,Atlanta,Asian,current,20534.83,7041.32,11582.98,0.78
2,Atlanta,Other,behind,8375.40,5765.38,9484.05,0.31
3,Atlanta,Other,current,18352.92,6439.09,10592.30,0.69
4,Atlanta,black,behind,84377.39,25286.83,41596.84,0.20
5,Atlanta,black,current,334560.43,52043.74,85611.95,0.80
6,Atlanta,white,behind,32461.67,13918.24,22895.50,0.08
7,Atlanta,white,current,361599.05,46427.68,76373.54,0.92


In [217]:
detail = full_crosstab(df1, ['WEEK','state','race','rent'], 'PWEIGHT', ['WEEK','state','race'], critical_val=1.645).round(2)
top = full_crosstab(df1, ['WEEK','state','race'], 'PWEIGHT', ['WEEK','state'], critical_val=1.645).round(2)
rv = detail.merge(top,'left',['WEEK','state','race'],suffixes=('_full','_demo'))
y = rv[rv.rent=='behind']
y_2 = y[y.proportions_full>0.4]
y_3 = y_2[y_2.PWEIGHT_full>y_2.mrgn_err_full]
y_3.state.unique()


array(['Alabama', 'Iowa', 'North Carolina', 'South Dakota', 'Virginia',
       'Illinois', 'Minnesota', 'New York', 'Ohio', 'Washington',
       'Connecticut', 'Indiana', 'New Mexico', 'Oregon', 'Georgia',
       'Louisiana', 'Maryland', 'Michigan', 'Pennsylvania', 'Wisconsin',
       'Alaska', 'District of Columbia', 'Nevada', 'Tennessee',
       'Mississippi', 'Nebraska', 'Oklahoma', 'Rhode Island',
       'South Carolina', 'Missouri', 'Texas', 'Colorado', 'Delaware',
       'Florida', 'Kentucky', 'Massachusetts', 'Vermont', 'California',
       'Kansas', 'New Jersey', 'Utah'], dtype=object)

In [209]:
detail_r = full_crosstab(df1, ['WEEK','region','race','rent'], 'PWEIGHT', ['WEEK','region','race'], critical_val=1.645).round(2)
top_r = full_crosstab(df1, ['WEEK','region','race'], 'PWEIGHT', ['WEEK','region'], critical_val=1.645).round(2)
rv_r = detail_r.merge(top_r,'left',['WEEK','region','race'],suffixes=('_full','_demo'))
x = rv_r[rv_r.rent=='behind']
x[x.proportions_full>0.4]

,WEEK,region,race,rent,PWEIGHT_full,std_err_full,mrgn_err_full,proportions_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions_demo
172,18,NE,black,behind,857991.48,189038.01,310967.52,0.44,1928187.96,195373.94,321390.14,0.20
292,22,MW,black,behind,733948.69,156260.04,257047.76,0.50,1479393.27,156888.39,258081.41,0.17
298,22,NE,Other,behind,396776.44,110644.16,182009.64,0.57,701300.10,117815.62,193806.69,0.08


In [ ]:
full_crosstab(df1, ['WEEK','region','race','rent'], 'PWEIGHT', ['WEEK','region','race'], critical_val=1.645).round(2)